In [1]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import matplotlib.pyplot as plt

In [2]:
def data_ret(degree=1,split_percent=70):
    
    pwd = os.getcwd()+'\\health_data.csv'
    dataset = pd.read_csv(pwd)             
    dataset = dataset.sample(frac = 1)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1:].values
    datasize = X.shape[0]
    split_point = split_percent//10
    X_train = X[:(datasize*split_point)//10,:]
    y_train = y[:(datasize*split_point)//10,:]
    X_test = X[(datasize*split_point)//10:,:]
    y_test = y[(datasize*split_point)//10:,:]



    X_train_mean = np.sum(X_train,axis=0).reshape(1,-1)/X_train.shape[0]
    X_train_var = np.sqrt(np.sum(np.square(X_train-X_train_mean),axis=0).reshape(1,-1)) / X_train.shape[0]
    X_train = (X_train-X_train_mean) / X_train_var
    X_test = (X_test-X_train_mean) / X_train_var
    

    X_train_ret,X_test_ret = X_train,X_test
    X_train_temp = X_train
    X_test_temp = X_test

    for i in range(2,degree+1):
        X_train_temp = np.multiply(X_train_temp,X_train)
        X_test_temp = np.multiply(X_test_temp,X_test)
        X_train_temp_var = np.sqrt(np.sum(np.square(X_train_temp),axis=0).reshape(1,-1)) / X_train.shape[0]

        X_train_temp = X_train_temp / (X_train_temp_var)
        X_test_temp = X_test_temp / (X_train_temp_var)

        X_train_ret = np.append(X_train_ret,X_train_temp,axis=1)
        X_test_ret = np.append(X_test_ret,X_test_temp,axis=1)

    X_train_ret = np.append(np.ones((X_train.shape[0],1)),X_train_ret,axis=1)
    X_test_ret = np.append(np.ones((X_test.shape[0],1)),X_test_ret,axis=1)

    return X_train_ret,y_train,X_test_ret,y_test

In [3]:
def data_ret2(split_percent=70):
    

    dataset = pd.read_excel('E:\ELL_Project\problem2\weather_data.xlsx')                     
    dataset = dataset.sample(frac = 1)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1:].values
    datasize = X.shape[0]
    split_point = split_percent//10
    X_train = X[:(datasize*split_point)//10,:]
    y_train = y[:(datasize*split_point)//10,:]
    X_test = X[(datasize*split_point)//10:,:]
    y_test = y[(datasize*split_point)//10:,:]
    return (X_train,X_test,y_train,y_test)

def datapol2(X_train,X_test,y_train,y_test,degree=1):

    X_train_mean = np.sum(X_train,axis=0).reshape(1,-1)/X_train.shape[0]
    X_train_var = np.sqrt(np.sum(np.square(X_train-X_train_mean),axis=0).reshape(1,-1)) / X_train.shape[0]
    X_train = (X_train-X_train_mean) / X_train_var
    X_test = (X_test-X_train_mean) / X_train_var
    

    X_train_ret,X_test_ret = X_train,X_test
    X_train_temp = X_train
    X_test_temp = X_test

    for i in range(2,degree+1):
        X_train_temp = np.multiply(X_train_temp,X_train)
        X_test_temp = np.multiply(X_test_temp,X_test)
        X_train_temp_var = np.sqrt(np.sum(np.square(X_train_temp),axis=0).reshape(1,-1)) / X_train.shape[0]

        X_train_temp = X_train_temp / (X_train_temp_var)
        X_test_temp = X_test_temp / (X_train_temp_var)

        X_train_ret = np.append(X_train_ret,X_train_temp,axis=1)
        X_test_ret = np.append(X_test_ret,X_test_temp,axis=1)

    X_train_ret = np.append(np.ones((X_train.shape[0],1)),X_train_ret,axis=1)
    X_test_ret = np.append(np.ones((X_test.shape[0],1)),X_test_ret,axis=1)

    return X_train_ret,y_train,X_test_ret,y_test

In [4]:
def gradDesc(X,y,theta,hypothesis,loss_function,regularizer,alpha,lambda1,lambda2):
    h,loss,gradient = loss_function(X,theta,y,hypothesis)
    if(regularizer==elastic_net_reg):
        reg_loss,reg_grad = regularizer(lambda1,lambda2,theta)
    else:
        reg_loss,reg_grad = regularizer(lambda1,theta)
    # loss += reg_loss
    gradient += reg_grad

    return(loss,gradient)

In [5]:
def mse_loss(X,theta,y,hypothesis):
    siz = y.shape[0]
    h = hypothesis(X,theta)
    diff = h-y
    mse = ( np.sum(np.square(diff),axis=0))[0]
    gradient = np.dot(np.transpose(X),diff) / siz
    return(h,mse,gradient)

def mae_loss(X,theta,y,hypothesis):
    siz = y.shape[0]
    num_feat = y.shape[1]
    h = hypothesis(X,theta)
    diff = h-y
    diff_sign = np.ones((siz,1))
    diff_sign[diff[:,0]<0] = -1
    mae = np.sum(np.abs(h),axis=0)[0] / siz
    gradient = np.sum(diff_sign*X,axis=0).reshape(-1,1)
    return(h,mae,gradient)

In [28]:
def lin_hyp(X,theta):
    # print(np.dot(X,theta).shape)
    return (np.dot(X,theta))
def log_hyp(X,theta):
    a = lin_hyp(X,theta)
    return ( 1/(1+np.exp(-a)) )
def null_regularizer(alpha,theta):
    a = np.zeros(theta.shape)
    return(0,a)
def l1_reg(alpha,theta):
    reg_loss = alpha*theta
    reg_grad = alpha
    return(reg_loss,reg_grad)
def l2_reg(alpha,theta):
    reg_loss = alpha * np.square(theta)
    reg_grad = 2 * alpha * theta
    return(reg_loss,reg_grad)
def elastic_net_reg(lambda1,lambda2,theta):
    a1,a2 = l1_reg(lambda1,theta)
    b1,b2 = l2_reg(lambda2,theta)
    return( a1+b1 , a2+b2 )

In [29]:
def linReg(X,y,iter=1000,alpha=0.01,batchSize=32,hypothesis = lin_hyp,loss_function=mse_loss,regularizer=l2_reg,lambda1 = 0.01,lambd2_elasnet = 0.01):
    
    theta = np.random.random((X.shape[1],1))
    ret_thet = []
    # print(theta)
    # print(theta)
    datasize = X.shape[0]
    loss_list = []
    loss_epoch,_ = gradDesc(X,y,theta,hypothesis,loss_function,regularizer,alpha,lambda1, lambd2_elasnet)
    for i in range(iter):
        if((i+1)%1000==0):
            loss_list.append(loss_epoch)
            print('Loss for {} iterations: {}'.format(i+1,loss_epoch))
        fro = 0
        loss_epoch = 0
        while(True):
            to = min(fro+batchSize,datasize)
            l,theta_grad = gradDesc(X[fro:to,:],y[fro:to,:],theta,hypothesis,loss_function,regularizer,alpha,lambda1, lambd2_elasnet)
            # print(theta_grad)
            loss_epoch += l
            
            # print(theta_grad)
            theta -= (alpha*theta_grad)
            ret_thet.append(theta)
            ll,_ = gradDesc(X,y,theta,hypothesis,loss_function,regularizer,alpha,lambda1, lambd2_elasnet)
            loss_list.append(ll)
            fro = to

            if(to>=datasize):
                break
        
    # print(ret_thet)
    return (theta,loss_epoch,loss_list,ret_thet)

In [30]:
def accuracy_metrics(y_pred,y_test):
    loss_y = y_pred - y_test

    test_size = y_pred.shape[0]
    total_loss_y = np.dot(np.ones((1,test_size)),np.square(loss_y))[0,0] / test_size

    # print(total_loss_y)

    y_pred_thresh = y_pred>=0.5

    tp = np.sum((y_pred_thresh+y_test)==2 , axis=0)[0]
    tn = np.sum(y_pred_thresh==y_test , axis=0)[0] - tp
    fp = np.sum(y_pred_thresh , axis=0)[0]-tp
    fn = test_size-tp-tn-fp


    print('tp: {} , tn: {} , fp: {} , fn: {}'.format(tp,tn,fp,fn))

    acc = (tp+tn)/test_size
    prec = (tp)/(tp+fp)
    recl = (tp)/(tp+fn)
    f1 = 2*prec*recl/(prec+recl)

    print('Accuracy: {}'.format( acc  ))
    print('Precision: {}'.format( prec  ))
    print('Recall: {}'.format( recl  ))
    print('F1 score: {}'.format( f1  ))

    return acc

In [34]:
X_train,y_train,X_test,y_test = data_ret(degree=5,split_percent=70)
train_datasize = X_train.shape[0]
opt_theta, train_loss , loss_list , ret_thet = linReg(X_train,y_train,iter=20000,alpha=0.0001,batchSize=10000,hypothesis = lin_hyp,loss_function=mse_loss,regularizer=l2_reg,lambda1=0.1,lambd2_elasnet = 0.001)
# y_pred_train = lin_hyp(X_train,opt_theta)
y_pred_test = lin_hyp(X_test,opt_theta)
# mse_tr,_ = accuracy(y_pred_train,y_train)
mse_te = accuracy_metrics(y_pred_test,y_test)

Loss for 1000 iterations: 384.4201381865984
Loss for 2000 iterations: 225.72543800384946
Loss for 3000 iterations: 149.07459453426546
Loss for 4000 iterations: 106.24531291185792
Loss for 5000 iterations: 81.29997933741092
Loss for 6000 iterations: 66.41967976475692
Loss for 7000 iterations: 57.422531419158666
Loss for 8000 iterations: 51.9443929374737
Loss for 9000 iterations: 48.600322176286156
Loss for 10000 iterations: 46.56107675945438
Loss for 11000 iterations: 45.32357762629897
Loss for 12000 iterations: 44.58020139314743
Loss for 13000 iterations: 44.14192885862866
Loss for 14000 iterations: 43.89225496874996
Loss for 15000 iterations: 43.75921393565872
Loss for 16000 iterations: 43.698276465866506
Loss for 17000 iterations: 43.6818476653655
Loss for 18000 iterations: 43.69280033671073
Loss for 19000 iterations: 43.720485840332834
Loss for 20000 iterations: 43.7582701059238
tp: 75 , tn: 108 , fp: 12 , fn: 15
Accuracy: 0.8714285714285714
Precision: 0.8620689655172413
Recall: 0.8